In [23]:
import yfinance as yf
import numpy as np
import pandas as pd

# Definir os tickers dos ativos
tickers = ['JPM', 'BBAS3.SA', 'KO', 'APA', 'CVS', 'UAL', 'DIS', 'SUZB3.SA']

# Baixar dados históricos de preços (fechamento)
data = yf.download(tickers, start="2016-01-01", end="2020-01-01")['Close']
data = data.dropna()  # Remover valores ausentes

# Calcular a matriz de covariância dos retornos dos ativos
cov_matrix = data.pct_change().cov()

# Função para otimização de Média-Variância
def mean_variance_optimization(expected_returns, cov_matrix):
    def objective(weights):
        port_return = np.dot(weights, expected_returns)
        port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        return -port_return / port_volatility  # Maximizar o Sharpe ratio
    
    n = len(expected_returns)
    init_guess = np.ones(n) / n  # Distribuição inicial igual
    bounds = [(0, 1) for _ in range(n)]
    constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}  # Restrição: soma dos pesos = 1

    result = minimize(objective, init_guess, method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

# Construção do Portfólio MFRP
weights_mfrp = mfrp_portfolio(cov_matrix)
print("MFRP Portfolio Weights:", weights_mfrp)

# Calcular o retorno esperado dos ativos (para otimização de média-variância)
expected_returns = data.mean()

# Otimização de Média-Variância
optimized_weights = mean_variance_optimization(expected_returns, cov_matrix)
print("Optimized Portfolio Weights (Mean-Variance):", optimized_weights)

# Simulando visões para o modelo Black-Litterman
tau = 0.05  # Constante de escala
sigma = cov_matrix  # Matriz de covariância dos ativos
P = np.array([[1, 0, 0, 0, 0, 0, 0, 0],  # Visão sobre o Ativo 1 (JPM)
              [0, 1, 0, 0, 0, 0, 0, 0]])  # Visão sobre o Ativo 2 (BBAS3.SA)
Q = np.array([0.02, -0.01])  # Visões sobre os ativos
omega = np.diag([0.0001, 0.0001])  # Incerteza das visões
Pi = expected_returns.values  # Visões de equilíbrio baseadas nos retornos esperados dos ativos

# Ajustando os retornos com o Black-Litterman
adjusted_returns = black_litterman(expected_returns.values, tau, sigma, P, Q, omega, Pi)
print("Adjusted Returns (Black-Litterman):", adjusted_returns)

# Calcular o momentum de série temporal para cada ativo
momentum = time_series_momentum(data)
print("Momentum:", momentum.tail())

# Combine as visões de Black-Litterman e Momentum para otimização
combined_returns = adjusted_returns + momentum.iloc[-1].values  # Combinando visões

# Otimizar novamente com os retornos ajustados combinados
final_weights = mean_variance_optimization(combined_returns, cov_matrix)
final_weights = np.round(final_weights, 2) * 100  # Arredondar e multiplicar por 100 para mostrar em %

# Exibir os pesos finais associados aos tickers
final_weights_df = pd.DataFrame(final_weights, index=tickers, columns=["Weight (%)"])
print("Final Portfolio Weights (Black-Litterman + Momentum):")
print(final_weights_df)


[*********************100%***********************]  8 of 8 completed

MFRP Portfolio Weights: [-0.0024346   0.0229894   0.06973383  0.17569702  0.091374    0.50124962
  0.12026236  0.02112836]
Optimized Portfolio Weights (Mean-Variance): [0.         0.         0.07048362 0.54111607 0.13766074 0.18743715
 0.0505423  0.0127779 ]
Adjusted Returns (Black-Litterman): [0.06945189 0.01532177 0.14904127 0.26271221 0.19235586 0.09161604
 0.06174251 0.17540136]
Momentum: Ticker           APA  BBAS3.SA       CVS       DIS       JPM        KO  \
Date                                                                     
2019-12-20  0.027615  0.114250  0.052682  0.017982  0.026972  0.040567   
2019-12-23  0.125905  0.158059  0.056058  0.017868  0.027183  0.041437   
2019-12-26  0.134753  0.132564  0.052047  0.020122  0.033696  0.030877   
2019-12-27  0.115533  0.153455  0.064203  0.021332  0.036975  0.026229   
2019-12-30  0.038893  0.144971  0.058575  0.015811  0.027402  0.034087   

Ticker      SUZB3.SA       UAL  
Date                            
2019-12-20  0.13658